### TEST - hur påverkas MSE och R2 av vilka features som används?
olika feature-kombinationer på tips-datasetet och skriver ut MSE och R² för varje.

In [ ]:
# experiment_linreg_tips_forloop.py
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = sns.load_dataset("tips").dropna(subset=["total_bill", "tip"])


y = df["tip"]

# 2) Definiera feature-set att testa (namn, kolumnlista)
feature_sets = [
    ("X1: total_bill", ["total_bill"]),
    ("X2: total_bill + size", ["total_bill", "size"]),
    ("X3: total_bill + size + time", ["total_bill", "size", "time"]),
    ("X4: total_bill + size + time + day", ["total_bill", "size", "time", "day"]),
    ("X5: total_bill + size + time + day + smoker", ["total_bill", "size", "time", "day", "smoker"]),
]

# 3) Bygg unionen av alla kolumner 
all_cols = set()
for name, cols in feature_sets:
    for c in cols:
        all_cols.add(c)

# Gör om till lista 
all_cols_list = list(all_cols)

# 4) Ta ut X_all med alla potentiella kolumner
X_all = df[all_cols_list].copy()

# 5) Samma train/test-split för ALLA jämförelser (rättvis jämförelse)
X_train_all, X_test_all, y_train, y_test = train_test_split(
    X_all, y, test_size=0.2, random_state=42
)

results = []

# 6) Loopa igenom varje feature-set och bygg/träna en pipeline per set
for name, cols in feature_sets:
    # 6a) Begränsa till detta sets kolumner
    X_train = X_train_all[cols].copy()
    X_test  = X_test_all[cols].copy()

    # 6b) Dela upp i numeriska/kategoriska (med for-loop-stil)
    num_cols = []
    cat_cols = []
    for col in X_train.columns:
        if pd.api.types.is_numeric_dtype(X_train[col]):
            num_cols.append(col)
        else:
            cat_cols.append(col)

    # 6c) Preprocess: StandardScaler för numeriska, OneHotEncoder för kategoriska
    numeric = Pipeline([("scaler", StandardScaler())])
    categorical = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore"))])  # sparse=True default

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric, num_cols),
            ("cat", categorical, cat_cols),
        ],
        remainder="drop"
    )

    pipe = Pipeline([
        ("preprocess", preprocessor),
        ("model", LinearRegression())
    ])

    # 6d) Träna och utvärdera
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2  = r2_score(y_test, y_pred)

    # 6e) Försök hämta feature-vikter (inte nödvändigt, men kul att se)
    try:
        feature_names = pipe.named_steps["preprocess"].get_feature_names_out()
        coefs = pipe.named_steps["model"].coef_
        # Plocka topp 5 i absolutvärde med “manuell” loop
        abs_coefs = [(i, abs(coef)) for i, coef in enumerate(coefs)]
        # sortera (manuellt) på absolutvärde, störst först
        abs_coefs_sorted = sorted(abs_coefs, key=lambda t: t[1], reverse=True)
        top_texts = []
        count = 0
        for i, _ in abs_coefs_sorted:
            top_texts.append(f"{feature_names[i]}={coefs[i]:.3f}")
            count += 1
            if count == 5:
                break
        top_feats = ", ".join(top_texts)
    except Exception:
        top_feats = "(feature-vikter utlämnade)"

    results.append((name, mse, r2, top_feats))

# 7) Sortera resultat på R² (högst först) med for-loop-kompatibel stil
results_sorted = sorted(results, key=lambda x: x[2], reverse=True)

print("\n=== Resultat (sorterat på R², högst först) ===")
for name, mse, r2, tops in results_sorted:
    print(f"{name:<45} MSE: {mse:.4f}   R²: {r2:.4f}   Top-features: {tops}")

# Tips:
# - Lägre MSE och högre R² är bättre (jämförelse funkar då y är samma).
# - Om fler features inte förbättrar utfallet → de tillför troligen mest brus.
# - Lägg till/ta bort feature-set i listan ovan och kör igen.



=== Resultat (sorterat på R², högst först) ===
X1: total_bill                                MSE: 0.5688   R²: 0.5449   Top-features: num__total_bill=0.936
X2: total_bill + size                         MSE: 0.6486   R²: 0.4811   Top-features: num__total_bill=0.795, num__size=0.249
X3: total_bill + size + time                  MSE: 0.6535   R²: 0.4772   Top-features: num__total_bill=0.801, num__size=0.248, cat__time_Lunch=0.027, cat__time_Dinner=-0.027
X4: total_bill + size + time + day            MSE: 0.6656   R²: 0.4675   Top-features: num__total_bill=0.808, num__size=0.237, cat__day_Sat=-0.102, cat__day_Sun=0.072, cat__day_Fri=0.045
X5: total_bill + size + time + day + smoker   MSE: 0.6997   R²: 0.4402   Top-features: num__total_bill=0.830, num__size=0.219, cat__day_Fri=0.101, cat__smoker_No=0.096, cat__smoker_Yes=-0.096
